<a href="https://colab.research.google.com/github/pradhyumn/Survival-Rate-Analysis-Cancer-Outcomes/blob/main/mskcc_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In order to get the fastest predictions you need to enable GPUs for the notebook:**
* Navigate to Edit→Notebook Settings
* select GPU from the Hardware Accelerator drop-down
(https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=oM_8ELnJq_wd)

In [ ]:
!pip install catboost
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.4 MB/s eta 0:00:00


In [ ]:
!pip install scikit-survival

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
!pip install lifelines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 16.3 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=5448e7d7b27c15c1d72724b55dd1faee098f61aad487c664a347e3806654084d
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [ ]:
import pandas as pd
import numpy as np

from sksurv.datasets import load_gbsg2, load_veterans_lung_cancer
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import KFold, RepeatedKFold

import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from lifelines.utils import concordance_index
from sklearn.model_selection import train_test_split

import pandas as pd

# Set the option to display all columns (None means no limit)
pd.set_option('display.max_columns', None)

# Now, when you use df.head(), all columns will be displayed
# Replace 'your_file.csv' with the path to your actual CSV file
csv_file_path = '/content/data_nsclc.csv'
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
df['Is_dead'] = df['dead'].apply(lambda x: 1 if x else 0)
df.head()


,PATIENT_ID,AGE,MALE,WHITE,ASIAN,BLACK,SMOKER,SEQUENCING_YEAR,stop,dead,ANY_CURRENT_cytotoxic,ANY_CURRENT_immunotherapy,ANY_CURRENT_other,ANY_CURRENT_targeted,ANY_CURRENT_vegf,ANY_PREV_cytotoxic,ANY_PREV_immunotherapy,ANY_PREV_other,ANY_PREV_targeted,ANY_PREV_vegf,ANY_PRIOR_TX,STAGE 1,STAGE 2,STAGE 3,STAGE 4,STAGE_IV_DX,STAGE_I-III_NOPROG,STAGE_I-III_PROG,entry,progressed,DMETS_DX_ADRENAL,DMETS_DX_BONE,DMETS_DX_BRAIN,DMETS_DX_LIVER,DMETS_DX_LUNG,DMETS_DX_LYMPH,DMETS_DX_PLEURA,DMETS_DX_OTHER,ADENOCARCINOMA,SQUAMOUS,PDL1,HAS_PDL1,KRAS,HRAS,RET,MET,GNAQ,PTEN,KIT,EGFR,FGFR3,PDGFRA,ERBB2,TP53,NRAS,NOTCH1,GNA11,CTNNB1,FGFR2,PIK3CA,IDH1,BRAF,FGFR1,ALK,AKT1,stop_DMETS_DX_BRAIN,stop_DMETS_DX_LUNG,stop_DMETS_DX_LIVER,stop_DMETS_DX_BONE,dead_DMETS_DX_BRAIN,dead_DMETS_DX_LUNG,dead_DMETS_DX_LIVER,dead_DMETS_DX_BONE,Is_dead
0,P-0000012,58,0,1,0,0,1,2013,3645,0,0,0,0,0,0,1,0,0,0,0,1,False,False,True,False,0,0,1,42,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3603,1681,3603,3603,0,1,0,0,0
1,P-0000231,27,0,1,0,0,0,2014,515,1,1,0,0,0,1,1,0,0,0,1,1,False,False,True,False,0,0,1,108,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,407,39,91,407,0,1,1,0,1
2,P-0000527,57,0,0,0,1,0,2014,204,1,1,0,0,0,0,0,0,0,0,0,0,False,False,True,False,0,0,1,96,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,204,68,204,68,0,1,0,1,1
3,P-0000622,67,0,1,0,0,0,2014,230,1,1,0,0,0,0,0,0,0,0,0,0,False,False,True,False,0,0,1,21,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,230,104,230,204,0,1,0,1,1
4,P-0000689,56,1,1,0,0,0,2014,906,1,1,0,0,0,0,0,0,0,0,0,0,False,False,True,False,0,0,1,50,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,679,25,637,536,1,1,1,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7810 entries, 0 to 7809
Data columns (total 74 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   PATIENT_ID                 7810 non-null   object
 1   AGE                        7810 non-null   int64 
 2   MALE                       7810 non-null   int64 
 3   WHITE                      7810 non-null   int64 
 4   ASIAN                      7810 non-null   int64 
 5   BLACK                      7810 non-null   int64 
 6   SMOKER                     7810 non-null   int64 
 7   SEQUENCING_YEAR            7810 non-null   int64 
 8   stop                       7810 non-null   int64 
 9   dead                       7810 non-null   int64 
 10  ANY_CURRENT_cytotoxic      7810 non-null   int64 
 11  ANY_CURRENT_immunotherapy  7810 non-null   int64 
 12  ANY_CURRENT_other          7810 non-null   int64 
 13  ANY_CURRENT_targeted       7810 non-null   int64 
 14  ANY_CURR

In [ ]:
# categorize columns
demographiccols = ['AGE','MALE','WHITE','ASIAN','BLACK','SMOKER','SEQUENCING_YEAR']

txhxcols = set(['ANY_PRIOR_TX'])

tumorregcols = ['STAGE 1','STAGE 2','STAGE 3','STAGE 4',
                'STAGE_IV_DX','STAGE_I-III_NOPROG','STAGE_I-III_PROG','entry','progressed']

metcols = ['DMETS_DX_ADRENAL',
 'DMETS_DX_BONE',
 'DMETS_DX_BRAIN',
 'DMETS_DX_LIVER',
 'DMETS_DX_LUNG',
 'DMETS_DX_LYMPH',
 'DMETS_DX_PLEURA',
 'DMETS_DX_OTHER']

pathcols = ['Gleason','HAS_Gleason','ADENOCARCINOMA','SQUAMOUS','PDL1','HAS_PDL1','HR','HER2',
           'RECTAL','ASCENDING','CECUM','NONADENOCARCINOMA','MUCINOUS',
            'MSI_OR_dMMR','HAS_MSI_OR_dMMR']

labcols = ['MAX_CA15-3','CA15-3','HAS_CA15-3',
          'MAX_CEA','CEA','HAS_CEA',
          'MAX_PSA','PSA','HAS_PSA',
          'MAX_CA19-9','CA19-9','HAS_CA19-9']

common_genes = ['KRAS', 'HRAS', 'RET', 'MET', 'GNAQ',
                'PTEN', 'KIT', 'EGFR', 'FGFR3', 'PDGFRA',
                'ERBB2', 'TP53', 'NRAS', 'NOTCH1', 'GNA11',
                'CTNNB1', 'FGFR2', 'PIK3CA', 'IDH1', 'BRAF',
                'FGFR1', 'ALK', 'AKT1']

cancers_of_interest = ['nsclc','brca','prostate','crc','panc']

# # load 5 files into a dictionary of <cancer type>:<pandas dataframe>
# cancers_of_interest = ['nsclc','brca','prostate','crc','panc']
# cancer2df_master_current_tx = {}
# for c in cancers_of_interest:
#     cancer2df_master_current_tx[c] = pd.read_csv('data/'+c+'_dx_1st_seq_OS.csv')
#     columns_noID = list(cancer2df_master_current_tx[c].columns)[1:]
#     txhxcols_c = cancer2df_master_current_tx[c].columns[cancer2df_master_current_tx[c].columns.str.contains('ANY_CURRENT_|ANY_PREV_',regex=True)]
#     txhxcols = txhxcols.union(set(txhxcols_c))

In [ ]:
def run_cb(df_train, df_tests, cois, endsuffix='', **kwargs):
    cois = list(set(df_train.columns).intersection(set(cois)))
    #cois += ['Is_dead']
    if len(cois) > 0:
        df_train['stop'+endsuffix]=np.where(df_train['Is_dead'], df_train['stop'+endsuffix], - df_train['stop'+endsuffix])
        X_train = df_train[cois].fillna(0).astype(int)
        y_train = df_train['stop'+endsuffix].astype(float)  # Only use survival times

        model = CatBoostRegressor(loss_function='Cox', iterations=1000, learning_rate=0.01, depth=7, l2_leaf_reg=3.5, eval_metric='Cox', verbose=200, random_seed=42)
        model.fit(X_train, y_train, verbose=False)  # Fit model

        scores = []
        for df_test in df_tests:
            X_test = df_test[cois].fillna(0).astype(float)
            y_test = df_test[['stop'+endsuffix, 'dead'+endsuffix]].values
            predictions = model.predict(X_test)
            # Implement Concordance Index (C-index) correctly
            score = concordance_index(y_test[:, 0], -predictions, event_observed=y_test[:, 1])  # Assuming y_test is structured correctly
            scores.append(score)
        return scores
    return [np.nan] * len(df_tests)


In [ ]:
txhxcols = list(txhxcols)
all_cols = demographiccols+txhxcols+tumorregcols+metcols+pathcols+labcols+common_genes

# note: uncomment or comment items in variable_list and labels to include or exclude more categories of variables
variable_list = [demographiccols,
#                 txhxcols,
#                 tumorregcols,
#                 metcols,
#                 pathcols,
#                 labcols,
                 common_genes]
#                all_cols]

# for ablation studies
#variable_list = [set(all_cols)-set(v) for v in variable_list]

labels = ['Demographics',
#          'Treatment',
#          'Stage/Progression',
#          'Other Met Sites',
#          'Pathology',
#          'Tumor Markers',
          'Genomics']
#         'All']

scores = pd.DataFrame(columns=labels)

for c in ['nsclc']: #testing on 2/5 but can substitute for cancers_of_interest
    print(c)
    for i, variables in enumerate(variable_list):
        print(variables)
        dftemp = df
        dftemp['stop_nonlt'] = dftemp['stop']-dftemp['entry']
        dftemp['dead_nonlt']=dftemp['dead']
        dftemp = dftemp[dftemp['stop_nonlt']>=0].sample(frac=1).reset_index()
        kf = KFold(n_splits=5)# Define the split - into n_splits folds
        kf.get_n_splits(dftemp) # returns the number of splitting iterations in the cross-validator

        for f, (train_index, test_index) in enumerate(kf.split(dftemp)):
            print(f'fold: {f}')
            train = dftemp.loc[train_index]
            val = dftemp.loc[test_index]

            val_list = [val,val[val['STAGE_IV_DX'].astype(bool)],val[val['STAGE_I-III_NOPROG'].astype(bool)]]
            print([len(a) for a in val_list])
            scorelist = (run_cb(train,val_list,variables,'_nonlt'))
            prefix='fold'+str(f)+'_'
            scores.loc[prefix+c,labels[i]] = scorelist[0]
            scores.loc[prefix+'stage IV '+c,labels[i]] = scorelist[1]
            scores.loc[prefix+'stage I-III '+c,labels[i]] = scorelist[2]

scores.to_csv('test_cb_output.csv')

nsclc
['AGE', 'MALE', 'WHITE', 'ASIAN', 'BLACK', 'SMOKER', 'SEQUENCING_YEAR']
fold: 0
[1562, 692, 454]
fold: 1
[1562, 659, 453]
fold: 2
[1562, 649, 439]
fold: 3
[1562, 668, 414]
fold: 4
[1562, 659, 443]
['KRAS', 'HRAS', 'RET', 'MET', 'GNAQ', 'PTEN', 'KIT', 'EGFR', 'FGFR3', 'PDGFRA', 'ERBB2', 'TP53', 'NRAS', 'NOTCH1', 'GNA11', 'CTNNB1', 'FGFR2', 'PIK3CA', 'IDH1', 'BRAF', 'FGFR1', 'ALK', 'AKT1']
fold: 0
[1562, 661, 454]
fold: 1
[1562, 662, 430]
fold: 2
[1562, 708, 422]
fold: 3
[1562, 666, 442]
fold: 4
[1562, 630, 455]


In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
import xgboost as xgb

def run_xgb(df_train, df_tests, cois, endsuffix='', **kwargs):
    cois = list(set(df_train.columns).intersection(set(cois)))
    #cois += ['Is_dead']
    if len(cois) > 0:
        # Split features and labels
        df_train['Survival_label_lower_bound']=df_train['stop'+endsuffix]
        df_train['Survival_label_upper_bound'] = df_train.apply(
            lambda row: np.inf if row['Is_dead'] == 0 else row['stop'+endsuffix], axis=1)
        y_lower_bound = df_train['Survival_label_lower_bound']
        y_upper_bound = df_train['Survival_label_upper_bound']
        df_train = df_train.drop(['Survival_label_lower_bound', 'Survival_label_upper_bound'], axis=1)
        X_train = df_train[cois].fillna(0).astype(int)
        # Directly use survival times as the target variable
        #y_train = df_train['stop'+endsuffix].astype(float)

        dtrain = xgb.DMatrix(X_train.values)
        dtrain.set_float_info('label_lower_bound', y_lower_bound)
        dtrain.set_float_info('label_upper_bound', y_upper_bound)

        params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.20,
          'max_depth': 7,
          'lambda': 0.01,
          'alpha': 0.02}
        bst = xgb.train(params, dtrain, num_boost_round=10000,
                evals=[(dtrain, 'train')],
                early_stopping_rounds=50)

        scores = []
        for df_test in df_tests:
            X_test = df_test[cois].fillna(0).astype(float)
            dtest = xgb.DMatrix(X_test.values)
            y_test = df_test[['stop'+endsuffix, 'dead'+endsuffix]].values
            predictions = bst.predict(dtest)

            # Implement Concordance Index (C-index) for evaluation
            score = concordance_index(y_test[:, 0], predictions, event_observed=y_test[:, 1])
            scores.append(score)
        return scores
    return [np.nan] * len(df_tests)


In [ ]:
txhxcols = list(txhxcols)
all_cols = demographiccols+txhxcols+tumorregcols+metcols+pathcols+labcols+common_genes

# note: uncomment or comment items in variable_list and labels to include or exclude more categories of variables
variable_list = [demographiccols,
#                 txhxcols,
#                 tumorregcols,
#                 metcols,
#                 pathcols,
#                 labcols,
                 common_genes]
#                all_cols]

# for ablation studies
#variable_list = [set(all_cols)-set(v) for v in variable_list]

labels = ['Demographics',
#          'Treatment',
#          'Stage/Progression',
#          'Other Met Sites',
#          'Pathology',
#          'Tumor Markers',
          'Genomics']
#         'All']

scores = pd.DataFrame(columns=labels)

for c in ['nsclc']: #testing on 2/5 but can substitute for cancers_of_interest
    print(c)
    for i, variables in enumerate(variable_list):
        print(variables)
        dftemp = df
        dftemp['stop_nonlt'] = dftemp['stop']-dftemp['entry']
        dftemp['dead_nonlt']=dftemp['dead']
        dftemp = dftemp[dftemp['stop_nonlt']>=0].sample(frac=1).reset_index()
        kf = KFold(n_splits=5)# Define the split - into n_splits folds
        kf.get_n_splits(dftemp) # returns the number of splitting iterations in the cross-validator

        for f, (train_index, test_index) in enumerate(kf.split(dftemp)):
            print(f'fold: {f}')
            train = dftemp.loc[train_index]
            val = dftemp.loc[test_index]

            val_list = [val,val[val['STAGE_IV_DX'].astype(bool)],val[val['STAGE_I-III_NOPROG'].astype(bool)]]
            print([len(a) for a in val_list])
            scorelist = (run_xgb(train,val_list,variables,'_nonlt'))
            prefix='fold'+str(f)+'_'
            scores.loc[prefix+c,labels[i]] = scorelist[0]
            scores.loc[prefix+'stage IV '+c,labels[i]] = scorelist[1]
            scores.loc[prefix+'stage I-III '+c,labels[i]] = scorelist[2]

scores.to_csv('test_xgb_output.csv')

Streaming output truncated to the last 5000 lines.
[5000]	train-aft-nloglik:4.33339
[5001]	train-aft-nloglik:4.33339
[5002]	train-aft-nloglik:4.33339
[5003]	train-aft-nloglik:4.33339
[5004]	train-aft-nloglik:4.33338
[5005]	train-aft-nloglik:4.33338
[5006]	train-aft-nloglik:4.33338
[5007]	train-aft-nloglik:4.33338
[5008]	train-aft-nloglik:4.33338
[5009]	train-aft-nloglik:4.33338
[5010]	train-aft-nloglik:4.33338
[5011]	train-aft-nloglik:4.33338
[5012]	train-aft-nloglik:4.33338
[5013]	train-aft-nloglik:4.33338
[5014]	train-aft-nloglik:4.33338
[5015]	train-aft-nloglik:4.33338
[5016]	train-aft-nloglik:4.33338
[5017]	train-aft-nloglik:4.33338
[5018]	train-aft-nloglik:4.33338
[5019]	train-aft-nloglik:4.33338
[5020]	train-aft-nloglik:4.33338
[5021]	train-aft-nloglik:4.33338
[5022]	train-aft-nloglik:4.33338
[5023]	train-aft-nloglik:4.33338
[5024]	train-aft-nloglik:4.33338
[5025]	train-aft-nloglik:4.33338
[5026]	train-aft-nloglik:4.33338
[5027]	train-aft-nloglik:4.33338
[5028]	train-aft-nloglik:

In [ ]:
def runRF(df_train,df_tests,cois,endsuffix='',**kwargs):

    cois = list(set(df_train.columns).intersection(set(cois)))
    #cois += ['Is_dead']
    if len(cois)>0:
        X_train = df_train[cois].fillna(0).astype(int)
        y_train = df_train[['dead'+endsuffix,'stop'+endsuffix]].apply(tuple, axis=1).values.tolist()
        y_train = np.array(y_train, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

        random_state = 20
        rsf = RandomSurvivalForest(n_estimators=100,
                                   min_samples_split=10,
                                   min_samples_leaf=15,
                                   n_jobs=-1,
                                   random_state=random_state)
        rsf.fit(X_train, y_train)
        scores = []
        for df_test in df_tests:
            X_test = df_test[cois].fillna(0).astype(int)
            y_test = df_test[['dead'+endsuffix,'stop'+endsuffix]].apply(tuple, axis=1).values.tolist()
            y_test = np.array(y_test, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
            try:
                scores += [rsf.score(X_test, y_test)]
            except:
                scores += [np.nan]
        return scores
    return [np.nan]*len(df_tests)

In [ ]:
txhxcols = list(txhxcols)
all_cols = demographiccols+txhxcols+tumorregcols+metcols+pathcols+labcols+common_genes

# note: uncomment or comment items in variable_list and labels to include or exclude more categories of variables
variable_list = [demographiccols,
#                 txhxcols,
#                 tumorregcols,
#                 metcols,
#                 pathcols,
#                 labcols,
                 common_genes]
#                all_cols]

# for ablation studies
#variable_list = [set(all_cols)-set(v) for v in variable_list]

labels = ['Demographics',
#          'Treatment',
#          'Stage/Progression',
#          'Other Met Sites',
#          'Pathology',
#          'Tumor Markers',
          'Genomics']
#         'All']

scores = pd.DataFrame(columns=labels)

for c in ['nsclc']: #testing on 2/5 but can substitute for cancers_of_interest
    print(c)
    for i, variables in enumerate(variable_list):
        print(variables)
        dftemp = df
        dftemp['stop_nonlt'] = dftemp['stop']-dftemp['entry']
        dftemp['dead_nonlt']=dftemp['dead']
        dftemp = dftemp[dftemp['stop_nonlt']>=0].sample(frac=1).reset_index()
        kf = KFold(n_splits=5)# Define the split - into n_splits folds
        kf.get_n_splits(dftemp) # returns the number of splitting iterations in the cross-validator

        for f, (train_index, test_index) in enumerate(kf.split(dftemp)):
            print(f'fold: {f}')
            train = dftemp.loc[train_index]
            val = dftemp.loc[test_index]

            val_list = [val,val[val['STAGE_IV_DX'].astype(bool)],val[val['STAGE_I-III_NOPROG'].astype(bool)]]
            print([len(a) for a in val_list])
            scorelist = (runRF(train,val_list,variables,'_nonlt'))
            prefix='fold'+str(f)+'_'
            scores.loc[prefix+c,labels[i]] = scorelist[0]
            scores.loc[prefix+'stage IV '+c,labels[i]] = scorelist[1]
            scores.loc[prefix+'stage I-III '+c,labels[i]] = scorelist[2]

scores.to_csv('test_rf_output.csv')

nsclc
['AGE', 'MALE', 'WHITE', 'ASIAN', 'BLACK', 'SMOKER', 'SEQUENCING_YEAR']
fold: 0
[1562, 695, 421]
fold: 1
[1562, 656, 452]
fold: 2
[1562, 647, 461]
fold: 3
[1562, 656, 425]
fold: 4
[1562, 673, 444]
['KRAS', 'HRAS', 'RET', 'MET', 'GNAQ', 'PTEN', 'KIT', 'EGFR', 'FGFR3', 'PDGFRA', 'ERBB2', 'TP53', 'NRAS', 'NOTCH1', 'GNA11', 'CTNNB1', 'FGFR2', 'PIK3CA', 'IDH1', 'BRAF', 'FGFR1', 'ALK', 'AKT1']
fold: 0
[1562, 643, 458]
fold: 1
[1562, 677, 426]
fold: 2
[1562, 703, 418]
fold: 3
[1562, 645, 471]
fold: 4
[1562, 659, 430]


In [ ]:
x,y=load_veterans_lung_cancer()



In [ ]:
x.head()

,Age_in_years,Celltype,Karnofsky_score,Months_from_Diagnosis,Prior_therapy,Treatment
0,69.0,squamous,60.0,7.0,no,standard
1,64.0,squamous,70.0,5.0,yes,standard
2,38.0,squamous,60.0,3.0,no,standard
3,63.0,squamous,60.0,9.0,yes,standard
4,65.0,squamous,70.0,11.0,yes,standard


In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit

import xgboost as xgb

df = pd.read_csv('cancer.csv')
#print('Training data:')
#print(df)

# Split features and labels
df['Survival_label_lower_bound']=df['time']
df['Survival_label_upper_bound'] = df.apply(
    lambda row: np.inf if row['status'] == 1 else row['time'], axis=1)
# Split features and labels
y_lower_bound = df['Survival_label_lower_bound']
y_upper_bound = df['Survival_label_upper_bound']
X = df.drop(['Survival_label_lower_bound', 'Survival_label_upper_bound','inst','status'], axis=1)

# Split data into training and validation sets
rs = ShuffleSplit(n_splits=2, test_size=.7, random_state=0)
train_index, valid_index = next(rs.split(X))
dtrain = xgb.DMatrix(X.values[train_index, :])
dtrain.set_float_info('label_lower_bound', y_lower_bound[train_index])
dtrain.set_float_info('label_upper_bound', y_upper_bound[train_index])
dvalid = xgb.DMatrix(X.values[valid_index, :])
dvalid.set_float_info('label_lower_bound', y_lower_bound[valid_index])
dvalid.set_float_info('label_upper_bound', y_upper_bound[valid_index])

# Train gradient boosted trees using AFT loss and metric
params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.20,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}
bst = xgb.train(params, dtrain, num_boost_round=10000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=50)

# Run prediction on the validation set
df = pd.DataFrame({'Label (lower bound)': y_lower_bound[valid_index],
                   'Label (upper bound)': y_upper_bound[valid_index],
                   'Predicted label': bst.predict(dvalid)})
print(df)
# Show only data points with right-censored labels
#print(df[np.isinf(df['Label (upper bound)'])])

[0]	train-aft-nloglik:16.73606	valid-aft-nloglik:17.53614
[1]	train-aft-nloglik:15.57527	valid-aft-nloglik:16.31527
[2]	train-aft-nloglik:14.52626	valid-aft-nloglik:15.22299
[3]	train-aft-nloglik:13.57825	valid-aft-nloglik:14.23503
[4]	train-aft-nloglik:12.72163	valid-aft-nloglik:13.33191
[5]	train-aft-nloglik:11.94718	valid-aft-nloglik:12.52641
[6]	train-aft-nloglik:11.24706	valid-aft-nloglik:11.80099
[7]	train-aft-nloglik:10.61409	valid-aft-nloglik:11.13945
[8]	train-aft-nloglik:10.04184	valid-aft-nloglik:10.53308
[9]	train-aft-nloglik:9.52426	valid-aft-nloglik:9.99300
[10]	train-aft-nloglik:9.05617	valid-aft-nloglik:9.50266
[11]	train-aft-nloglik:8.63275	valid-aft-nloglik:9.05952
[12]	train-aft-nloglik:8.24942	valid-aft-nloglik:8.66224
[13]	train-aft-nloglik:7.90248	valid-aft-nloglik:8.30307
[14]	train-aft-nloglik:7.58850	valid-aft-nloglik:7.97526
[15]	train-aft-nloglik:7.30389	valid-aft-nloglik:7.67518
[16]	train-aft-nloglik:7.04634	valid-aft-nloglik:7.40389
[17]	train-aft-nloglik:

In [ ]:
import numpy as np
import xgboost as xgb

# 4-by-2 Data matrix
X = np.array([[1, -1], [-1, 1], [0, 1], [1, 0]])
dtrain = xgb.DMatrix(X)

# Associate ranged labels with the data matrix.
# This example shows each kind of censored labels.
#                         uncensored    right     left  interval
y_lower_bound = np.array([      2.0,     3.0,     0.0,     4.0])
y_upper_bound = np.array([      2.0, +np.inf,     4.0,     5.0])
dtrain.set_float_info('label_lower_bound', y_lower_bound)
dtrain.set_float_info('label_upper_bound', y_upper_bound)

params = {'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.20,
          'tree_method': 'hist', 'learning_rate': 0.05, 'max_depth': 2}
bst = xgb.train(params, dtrain, num_boost_round=5,
                evals=[(dtrain, 'train')])

[0]	train-aft-nloglik:2.30142
[1]	train-aft-nloglik:2.24184
[2]	train-aft-nloglik:2.18633
[3]	train-aft-nloglik:2.13462
[4]	train-aft-nloglik:2.08645


In [ ]:
bst.predict(dtrain)

array([0.6762048, 0.6762048, 0.6762048, 0.6762048], dtype=float32)

In [ ]:
dtrain